# ML Pipeline Preparation

In [ ]:
import nltk

nltk.download(['punkt', 'stopwords', 'wordnet'])

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import re
import joblib

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer



Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# load data from database
engine = create_engine('sqlite:///../data/processed/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
df.head()

In [ ]:
df.genre.value_counts()

In [ ]:
GenreType = pd.CategoricalDtype(categories=["news", "direct", "social"], ordered=False)
df.genre = df.genre.astype(GenreType)

df.info()

In [ ]:
df["genre_code"] = df.genre.cat.codes

In [ ]:
X = df[["message", "genre", "genre_code"]]
X.head()

In [ ]:
y = df.drop(columns=["id", "message", "genre", "original"])
# y = df[["storm", "fire", "earthquake", "cold"]]
y.head()

### 2. Write a tokenization function to process your text data

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words("english")
pattern = re.compile(r"[^a-zA-Z0-9]")

def tokenize(text):
    text = pattern.sub(" ", text.lower())
    tokens = word_tokenize(text)

    tokens = [lemmatizer.lemmatize(t).strip() for t in tokens if t not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [ ]:
pipeline = Pipeline([
    ("selector", ItemSelector("message")),
    ("count", CountVectorizer(tokenizer=tokenize)),
    ("vector", TfidfTransformer()),
    ("clf", MultiOutputClassifier(LinearSVC(random_state=42)))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
def get_metrics(y_test, y_pred):
    scores = []

    for i, col in enumerate(y.columns):
        report = classification_report(y_test.iloc[:, i], y_pred[:, i], output_dict=True, zero_division=0)

        scores.append({ 
            "category": col,
            "precision_0": report["0"]["precision"],
            "recall_0": report["0"]["recall"],
            "f1_0": report["0"]["f1-score"],
            "support_0": report["0"]["support"],
            "precision_1": report["1"]["precision"],
            "recall_1": report["1"]["recall"],
            "f1_1": report["1"]["f1-score"],
            "support_1": report["1"]["support"],
            "accuracy": report["accuracy"],
            "precision_macro_avg": report["macro avg"]["precision"],
            "recall_macro_avg": report["macro avg"]["recall"],
            "f1_macro_avg": report["macro avg"]["f1-score"],
            "support_macro_avg": report["macro avg"]["support"],
            "precision_weighted_avg": report["weighted avg"]["precision"],
            "recall_weighted_avg": report["weighted avg"]["recall"],
            "f1_weighted_avg": report["weighted avg"]["f1-score"],
            "support_weighted_avg": report["weighted avg"]["support"]
        })

    return pd.DataFrame.from_records(scores)

In [ ]:
get_metrics(y_test, y_pred).mean()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    "clf__estimator": [
        RandomForestClassifier(n_jobs=8),
        LinearSVC(),
        ExtraTreesClassifier(n_jobs=8)
    ]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

In [ ]:

cv.fit(X_train, y_train)

print("Best Parameters:", cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

get_metrics(y_test, y_pred).mean()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class DataFrameToArrayTransformer(BaseEstimator, TransformerMixin):    

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return X.to_numpy()

In [ ]:
pipeline = Pipeline([
    ("features", ColumnTransformer([
        ("genre_category", OneHotEncoder(dtype=int), ["genre"]),
        ("text", Pipeline([
            ("count", CountVectorizer(tokenizer=tokenize)),
            ("tfidf", TfidfTransformer())
        ]), "message")
    ], remainder="drop")),
    ("clf", MultiOutputClassifier(LinearSVC(random_state=42)))
])


In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
get_metrics(y_test, y_pred).mean()

### 9. Export your model as a pickle file

In [ ]:
joblib.dump(cv.best_estimator_, "../models/disaster_response_model.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.